In [55]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers.json import SimpleJsonOutputParser
from langchain_core.output_parsers.string import StrOutputParser
from langchain.output_parsers.boolean import BooleanOutputParser
from tqdm.auto import tqdm
from pydantic import BaseModel, Field

In [4]:
import os
from getpass import getpass

# Set the OpenAI API key
# You can set the key directly or use an environment variable
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

In [43]:
from bs4 import BeautifulSoup
import requests

url = 'https://sv.pokedb.tokyo/trainer/list?season=27&rule=0&name=&party=1&page=1'
response = requests.get(url)
response.encoding = response.apparent_encoding  # 文字エンコーディングを自動検出して設定
html = response.text

soup = BeautifulSoup(html, 'html.parser')
trainer_rows = soup.select('tbody tr')

trainers_data = []

for row in trainer_rows:
    rank = row.find('td', class_='has-text-weight-bold').text.strip().replace('\n\n\n構築記事', '')
    rating = row.find('td', class_='is-hidden-mobile').text.strip()
    trainer_name = row.find('div', class_='trainer-name').find_all('span')[-1].text.strip()
    
    article_link_tag = row.find('a', class_='link-team-article')
    article_link = article_link_tag['href'] if article_link_tag else None

    if article_link is None:
        continue
    
    trainers_data.append({
        '順位': int(rank),
        'レーティング': int(rating),
        'トレーナー名': trainer_name,
        '構築記事へのリンク': article_link
    })


In [41]:
from bs4 import BeautifulSoup
import requests

url = 'https://sv.pokedb.tokyo/trainer/list?season=27&rule=0&name=&party=1&page=2'
response = requests.get(url)
response.encoding = response.apparent_encoding  # 文字エンコーディングを自動検出して設定
html = response.text

soup = BeautifulSoup(html, 'html.parser')
trainer_rows = soup.select('tbody tr')

trainers_data2 = []

for row in trainer_rows:
    rank = row.find('td', class_='has-text-weight-bold').text.strip().replace('\n\n\n構築記事', '')
    rating = row.find('td', class_='is-hidden-mobile').text.strip()
    trainer_name = row.find('div', class_='trainer-name').find_all('span')[-1].text.strip()
    
    article_link_tag = row.find('a', class_='link-team-article')
    article_link = article_link_tag['href'] if article_link_tag else None

    if article_link is None:
        continue
    
    trainers_data2.append({
        '順位': int(rank),
        'レーティング': int(rating),
        'トレーナー名': trainer_name,
        '構築記事へのリンク': article_link
    })


In [44]:
trainers_data.extend(trainers_data2)

In [46]:
len(trainers_data)

163

In [83]:
model1 = ChatOpenAI(
    model="gpt-4o",
)

class PokemonInfo(BaseModel):
    name: str = Field(description="ポケモン名")
    item: str = Field(description="アイテム")
    nature: str = Field(description="性格")
    ability: str = Field(description="特性")
    Ttype: str = Field(description="テラスタルタイプ")
    moves: list[str] = Field(description="技")
    effort: list[int] = Field(description="努力値")

class ExtractedPokemons(BaseModel):
    internal_thinking_process: str = Field(description="思考過程")
    pokemons: list[PokemonInfo] = Field(description="ポケモンの情報")

model2 = ChatOpenAI(
    model="gpt-4o",
).with_structured_output(ExtractedPokemons)

prompt0 = ChatPromptTemplate.from_template(
    "あなたはポケモンに詳しいポケモンマスターです。"
    "以下のブログ記事の中にはポケモンに関する次の情報が含まれています。"
    "ポケモン名、タイプ、特性、技4つ、アイテム、性格、努力値、テラスタルタイプ"
    "この情報が全部で6体分存在します。"
    "ポケモン名: pokemon_name, "
    "特性: ability, "
    "技: moves, "
    "アイテム: item, "
    "性格: nature, "
    "努力値: evs(H,A,B,C,D,S), "
    "テラスタルタイプ: terastal_type(基本的にひらがな、カタカナで記載してください)"
    "重要！これらの情報が含まれているかどうかを確認してください。"
    "もし情報が抽出できそうなら YES, できなそうなら NO と答えてください。"
    "対象のブログ記事: {blog_article_content}"
)

prompt1 = ChatPromptTemplate.from_template(
    "あなたはポケモンに詳しいポケモンマスターです。"
    "以下のブログ記事の中にはポケモンに関する次の情報が含まれています。"
    "ポケモン名、タイプ、特性、技4つ、アイテム、性格、努力値、テラスタルタイプ"
    "この情報が全部で6体分存在します。それを次の形式で列挙してください。"
    "もし情報が不足している場合は、わかる範囲で記載してください。"
    "ポケモン名: pokemon_name, "
    "特性: ability, "
    "技: moves, "
    "アイテム: item, "
    "性格: nature, "
    "努力値: evs(H,A,B,C,D,S), "
    "なお、努力値ですが、努力値ではなく実数値で表記されている場合があります。例えば、131(4)-x-75-187(252)-155-205(252+) のような形です。"
    "この場合、努力値は [4, 0, 0, 252, 0, 252] としてください。"
    "テラスタルタイプ: terastal_type(基本的にひらがな、カタカナで記載してください)"
    "対象のブログ記事: {blog_article_content}"
)

prompt2 = ChatPromptTemplate.from_template(
    "あなたはポケモンに詳しいポケモンマスターです。"
    "これからあなたにはポケモンの情報6体分を提供します。"
    "そこから以下の情報を抽出してください。"
    "pokemon1_name,pokemon1_item,pokemon1_nature,pokemon1_ability,pokemon1_Ttype,pokemon1_moves,pokemon1_effort"
    "注意点を述べます。"
    "1. 漢字を使わず、ひらがなとカタカナのみで表記してください。"
    "2. 努力値は、H,A,B,C,D,Sの順番でカンマ区切りで記載してください。またここはシングルクォートで囲んでください。"
    "3. 技名は、カンマ区切りで4つ記載してください。またここはシングルクォートで囲んでください。漢字を使わず、ひらがなとカタカナのみで表記してください。"
    "いくつか注意すべき具体例を挙げます。"
    "突撃チョッキ→とつげきチョッキ、連撃ウーラオス→ウーラオス(れんげき)、暁ガチグマ→ガチグマ(アカツキ)、などです。"
    "出力の形式は次のようになります。json形式で出力してください。コードブロック表記は必要ありません。"
    "internal_thinking_process: str"
    "pokemons: list of dict(pokemon)"
    "pokemonsの中身は、以下のような json です。"
    "name: str, "
    "item: str, "
    "nature: str, "
    "ability: str, "
    "Ttype: str, "
    "moves: list of str, "
    "effort: list of int"
    "6体のポケモンの情報: {pokemon_info}"
)

In [84]:
# バトメモ画像で済まされている場合はスキップするようにしないといけない

extracted_data = []
skip_trainers = []

for trainer in tqdm(trainers_data):
    blog_url = trainer['構築記事へのリンク']
    if "yakkun" in blog_url:
        print(f"{trainer['順位']}位: ポケ徹記事のためスキップします。")
        skip_trainers.append(trainer)
        continue

    loader = WebBaseLoader(web_paths=[blog_url])
    docs = []
    async for doc in loader.alazy_load():
        docs.append(doc)
    doc = docs[0]
    boolean_chain = prompt0 | model1 | BooleanOutputParser()
    try:
        res = boolean_chain.invoke(
            {
                "blog_article_content": doc.page_content,
            },
        )
    except Exception as e:
        print(f"Error processing {trainer['順位']}位: {e}")
        skip_trainers.append(trainer)
        continue

    if res == False:
        print(f"{trainer['順位']}位: スキップします")
        skip_trainers.append(trainer)
        continue

    seq_chain = {"pokemon_info": prompt1 | model1 | StrOutputParser()} | prompt2 | model2
    res = seq_chain.invoke(
        {
            "blog_article_content": doc.page_content,
        },
    )
    extracted_data.append({
        "rank": trainer['順位'],
        "rating": trainer['レーティング'],
        "trainer_name": trainer['トレーナー名'],
        "blog_url": trainer['構築記事へのリンク'],
        "pokemon_info": res.pokemons,
    })

  0%|          | 0/163 [00:00<?, ?it/s]

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  3.28it/s]


24位: スキップします


Fetching pages: 100%|##########| 1/1 [00:00<00:00,  2.44it/s]


29位: スキップします


...

Fetching pages: 100%|##########| 1/1 [00:00<00:00, 14.80it/s]


852位: ポケ徹記事のためスキップします。
861位: ポケ徹記事のためスキップします。


Fetching pages: 100%|##########| 1/1 [00:00<00:00,  5.51it/s]


886位: ポケ徹記事のためスキップします。


Fetching pages: 100%|##########| 1/1 [00:00<00:00,  2.90it/s]


In [69]:
len(skip_trainers)

38

In [75]:
extracted_data[40]

{'rank': 127,
 'rating': 2008,
 'trainer_name': 'ピロスケ',
 'blog_url': 'https://pillosuke.hatenablog.com/entry/2025/03/02/035004',
 'pokemon_info': [PokemonInfo(name='エーフィ', item='ふうせん', nature='ひかえめ', ability='不明', Ttype='フェアリー', moves=['サイコノイズ', 'みわくのボイス', 'みらいよち', 'あさのひざし'], effort=[84, 0, 0, 252, 0, 172]),
  PokemonInfo(name='コライドン', item='とつげきチョッキ', nature='いじっぱり', ability='不明', Ttype='ほのお', moves=['フレアドライブ', 'ドレインパンチ', 'ニトロチャージ', 'こおりのキバ'], effort=[236, 196, 4, 0, 12, 60]),
  PokemonInfo(name='ハバタクカミ', item='おんみつマント', nature='おくびょう', ability='不明', Ttype='ひこう', moves=['ムーンフォース', 'めいそう', 'ちょうはつ', 'いたみわけ'], effort=[4, 0, 252, 0, 0, 252]),
  PokemonInfo(name='イーユイ', item='こだわりメガネ', nature='ひかえめ', ability='不明', Ttype='ほのお', moves=['かえんほうしゃ', 'あくのはどう', 'オーバーヒート', 'サイコキネシス'], effort=[228, 0, 4, 212, 60, 4]),
  PokemonInfo(name='ドオー', item='オボンのみ', nature='しんちょう', ability='不明', Ttype='でんき', moves=['じしん', 'どくどく', 'じこさいせい', 'ステルスロック'], effort=[252, 0, 4, 0, 252, 0]),
  PokemonInfo(name='ヘイラッ

In [85]:
# csv に変換する
import pandas as pd

flat_data_list = []
for i, trainer in enumerate(extracted_data, 1):
    flat_data = {}
    for i, pokemon in enumerate(trainer["pokemon_info"], 1):
        if i > 6:
            break

        prefix = f"pokemon{i}"
        flat_data[f"{prefix}_name"] = pokemon.name
        flat_data[f"{prefix}_item"] = pokemon.item
        flat_data[f"{prefix}_nature"] = pokemon.nature
        flat_data[f"{prefix}_ability"] = pokemon.ability
        flat_data[f"{prefix}_Ttype"] = pokemon.Ttype
        flat_data[f"{prefix}_moves"] = ", ".join(pokemon.moves)
        flat_data[f"{prefix}_effort"] = ", ".join(map(str, pokemon.effort)) if pokemon.effort else None

    flat_data_list.append({
        "rank": trainer["rank"],
        "rating": trainer["rating"],
        "trainer_name": trainer["trainer_name"],
        "blog_url": trainer["blog_url"],
        **flat_data
    })

df = pd.DataFrame(flat_data_list)

In [86]:
df

,rank,rating,trainer_name,blog_url,pokemon1_name,pokemon1_item,pokemon1_nature,pokemon1_ability,pokemon1_Ttype,pokemon1_moves,...,pokemon5_Ttype,pokemon5_moves,pokemon5_effort,pokemon6_name,pokemon6_item,pokemon6_nature,pokemon6_ability,pokemon6_Ttype,pokemon6_moves,pokemon6_effort
0,1,2174,アスノヨゾラ,https://orion-poke.hatenablog.com/entry/2025/0...,ホウオウ,ゴツゴツメット,わんぱく,さいせいりょく,フェアリー,"せいなるほのお, おにび, ひかりのかべ, じこさいせい",...,ゴースト,"アイアンヘッド, ちょうはつ, きりばらい, はねやすめ","252, 0, 252, 0, 4, 0",アローラベトベトン,くろいヘドロ,しんちょう,どくしゅ,どく,"どくづき, はたきおとす, かげうち, まもる","252, 0, 4, 0, 252, 0"
1,5,2136,ビッパ,https://bippa-poke.hatenablog.com/entry/2025/0...,ミライドン,とつげきチョッキ,不明,不明,こおり,"イナズマドライブ, りゅうせいぐん, テラバースト, ボルトチェンジ",...,ほのお,"ドレインパンチ, サンダーダイブ, れいとうパンチ, つるぎのまい","108, 116, 252, 0, 4, 28",ウーラオス(れんげき),パンチグローブ,不明,不明,ノーマル,"すいりゅうれんだ, ドレインパンチ, アクアジェット, つるぎのまい","204, 196, 100, 0, 4, 4"
2,7,2128,スカーレット,https://mega-salamance.hatenablog.com/entry/20...,ルナアーラ,こだわりスカーフ,ひかえめ,ファントムガード,フェアリー,"シャドーレイ, ムーンフォース, トリック, つきのひかり",...,みず,"しおづけ, まもる, みがわり, じこさいせい","252, 4, 156, 0, 52, 44",ほのおオーガポン,かまどのめん,いじっぱり,おもかげやどし,ほのお,"ツタこんぼう, ウッドホーン, がんせきふうじ, でんこうせっか","236, 204, 4, 0, 4, 60"
3,10,2121,ヘルメッポ,https://note.com/pokemons2/n/n054de1215fff,ディンルー,オボンのみ,わんぱく,わざわいのうつわ,みず,"じしん, まきびし, ステルスロック, ふきとばし",...,ステラ,"ゴールドラッシュ, シャドーボール, １０まんボルト, トリック","212, 0, 4, 60, 4, 228",ヘイラッシャ,ゴツゴツメット,わんぱく,てんねん,フェアリー,"じわれ, あくび, まもる, ねむる","252, 0, 252, 0, 4, 0"
4,19,2088,キャル,https://irohasu28.hatenadiary.com/entry/2025/0...,バドレックス(くろ),きあいのタスキ,おくびょう,じんばいったい,フェアリー,"アストラルビット, サイコショック, リーフストーム, わるだくみ",...,はがね,"ムーンフォース, あまえる, めいそう, いたみわけ","84, 0, 252, 132, 20, 20",ウーラオス(すい),とつげきチョッキ,いじっぱり,ふかしのこぶし,フェアリー,"すいりゅうれんだ, インファイト, テラバースト, アクアジェット","188, 252, 20, 0, 4, 44"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,919,1862,アルトマーレ,https://hayate-hyphen.hatenablog.com/entry/202...,ムゲンダイナ,こだわりメガネ,ひかえめ,プレッシャー,ほのお,"ダイマックスほう, りゅうせいぐん, ヘドロばくだん, かえんほうしゃ",...,フェアリー,"じしん, ゆきなだれ, ねむる, あくび","252, 0, 252, 0, 4, 0",ランドロス(れいじゅう),とつげきチョッキ,ゆうかん,いかく,みず,"じしん, じわれ, がんせきふうじ, とんぼがえり","212, 44, 0, 0, 252, 0"
123,926,1861,あいくま,https://note.com/aikumaaan/n/n83f6402fcb4d,バドレックス(クロウマ),こだわりメガネ,おくびょう,じんばいったい,フェアリー,"アストラルビット, サイコキネシス, リーフストーム, テラバースト",...,あく,"クイックターン, じしん, はたきおとす, ミラーコート","252, 0, 4, 0, 252, 0",オーガポン(ほのお),かまどのめん,いじっぱり,かたやぶり,ほのお,"ツタこんぼう, ウッドホーン, でんこうせっか, つるぎのまい","252, 228, 4, 0, 4, 20"
124,940,1860,(ENG),https://attem-ptpk.hatenablog.com/entry/2025/0...,テラパゴス,たべのこし,ひかえめ,テラスシェル,ステラ,"テラクラスター, まもる, めいそう, みがわり",...,ノーマル,"しんそく, じしん, りゅうのまい, はねやすめ","244, 180, 4, 0, 60, 20",ハバタクカミ,きあいのタスキ,おくびょう,こだいかっせい,ゴースト,"ムーンフォース, たたりめ, でんじは, ほろびのうた","4, 0, 0, 252, 0, 252"
125,959,1859,いくまろ,https://note.com/ymik0820/n/nd40fa14c8eef,バドレックス(くろ),こだわりメガネ,おくびょう,じんばいったい,フェアリー,,...,ノーマル,"ドラムアタック, はたきおとす, グラススライダー, とんぼがえり","132, 116, 4, 0, 252, 0",パオジアン,きあいのタスキ,いじっぱり,わざわいのつるぎ,でんき,"つららおとし, テラバースト, ふいうち, つるぎのまい","0, 252, 4, 0, 0, 252"


In [87]:
df.to_csv("auto_extracted_pokemon_data.csv", index=False)